In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from sklearn.utils import class_weight
import os
import itertools


In [2]:
train_dir = './datasets/train/'
validate_dir = './datasets/validate/'
datasample_period = 60
prediction_period = 10
feature_columns = 40
band_size = 0.001
temp = 200
n_train_files = len(os.listdir(train_dir))
n_validate_files = len(os.listdir(validate_dir))

In [3]:
def generate_data(directory, sample_size, prediction_period, feature_num, band_size):
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith((".npy")):
                data = np.load(os.path.join(subdir, file))[:temp]
                # generate X, Y
                shape = data.shape
                X = np.zeros((shape[0]-sample_size, sample_size, feature_num), dtype=np.float16)
                Y = np.zeros(shape=(shape[0]-sample_size, 1), dtype=np.int)
                for i in range(shape[0]-sample_size):
                    # take the first feature_num columns as features
                    X[i] = data[i:i+sample_size, 1:feature_num+1]
                    delta_last = (data[i+sample_size-1, 0] - data[i, 0]) / data[i+sample_size-1, 0]
                    if delta_last < -band_size:
                        Y[i] = 0
                    elif delta_last > band_size:
                        Y[i] = 2
                    else:
                        Y[i] = 1
                # add the 4th dimension: 1 channel
                X = X.reshape(X.shape[0], sample_size, feature_num, 1)
                
                # calculate sample_weights for Y
                sample_weights_y = np.append(Y.flatten(), [0,1,2]) # to ensure exhaustive coverage
                sample_weights_categories = class_weight.compute_class_weight('balanced', [0,1,2], sample_weights_y)
                idx = 0
                sample_weights = np.zeros(shape[0]-sample_size)
                for y in Y.flatten(): 
                    sample_weights[idx] = sample_weights_categories[y]
                    idx += 1

                # transform y to categorical arrays
                y_labels = to_categorical(sample_weights_y)[:-3]

                yield X, y_labels, sample_weights

In [4]:
batch = generate_data(train_dir, datasample_period, prediction_period, feature_columns, band_size)

In [5]:
dataset = next(batch)
dataset

(array([[[[3.7201e-02],
          [4.0750e+01],
          [3.7262e-02],
          ...,
          [3.2031e+01],
          [3.7598e-02],
          [1.4602e+01]],
 
         [[3.7201e-02],
          [4.0812e+01],
          [3.7262e-02],
          ...,
          [3.2031e+01],
          [3.7598e-02],
          [1.4602e+01]],
 
         [[3.7201e-02],
          [3.6344e+01],
          [3.7262e-02],
          ...,
          [3.0578e+01],
          [3.7598e-02],
          [1.4391e+01]],
 
         ...,
 
         [[3.7201e-02],
          [4.8344e+01],
          [3.7262e-02],
          ...,
          [3.0594e+01],
          [3.7598e-02],
          [1.4555e+01]],
 
         [[3.7170e-02],
          [8.0875e+01],
          [3.7262e-02],
          ...,
          [3.8656e+01],
          [3.7598e-02],
          [2.9266e+01]],
 
         [[3.7170e-02],
          [8.7875e+01],
          [3.7262e-02],
          ...,
          [3.8625e+01],
          [3.7598e-02],
          [2.9922e+01]]],
 
 
        [

In [6]:
dataset[0].shape

(140, 60, 40, 1)

In [7]:
dataset[1].shape

(140, 3)

In [8]:
dataset[2].shape

(140,)

In [9]:
input_tensor = Input(shape=(datasample_period,feature_columns,1))

# convolutional filter is (1,2) with stride of (1,2)
layer_x = layers.Conv2D(16, (1,2), strides=(1,2))(input_tensor)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

layer_x = layers.Conv2D(16, (1,2), strides=(1,2))(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

layer_x = layers.Conv2D(16, (1,10))(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

# Inception Module
tower_1 = layers.Conv2D(32, (1,1), padding='same')(layer_x)
tower_1 = layers.LeakyReLU(alpha=0.01)(tower_1)
tower_1 = layers.Conv2D(32, (3,1), padding='same')(tower_1)
tower_1 = layers.LeakyReLU(alpha=0.01)(tower_1)

tower_2 = layers.Conv2D(32, (1,1), padding='same')(layer_x)
tower_2 = layers.LeakyReLU(alpha=0.01)(tower_2)
tower_2 = layers.Conv2D(32, (5,1), padding='same')(tower_2)
tower_2 = layers.LeakyReLU(alpha=0.01)(tower_2)  

tower_3 = layers.MaxPooling2D((3,1), padding='same', strides=(1,1))(layer_x)
tower_3 = layers.Conv2D(32, (1,1), padding='same')(tower_3)
tower_3 = layers.LeakyReLU(alpha=0.01)(tower_3)

layer_x = layers.concatenate([tower_1, tower_2, tower_3], axis=-1)

# concatenate features of tower_1, tower_2, tower_3
layer_x = layers.Reshape((datasample_period,96))(layer_x)

# 64 LSTM units
layer_x = LSTM(64)(layer_x)
# The last output layer uses a softmax activation function
output = layers.Dense(3, activation='softmax')(layer_x)
model = Model(input_tensor, output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 60, 40, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 60, 20, 16)   48          input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 60, 20, 16)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 60, 20, 16)   1040        leaky_re_lu[0][0]                
______________________________________________________________________________________________

In [10]:
opt = tf.keras.optimizers.Adam(lr=0.01, epsilon=1) # learning rate and epsilon are the same as paper DeepLOB
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

train_generator = generate_data(train_dir, datasample_period, prediction_period, feature_columns, band_size)
validate_generator = generate_data(validate_dir, datasample_period, prediction_period, feature_columns, band_size)

model.fit(itertools.cycle(train_generator), epochs=100, steps_per_epoch=n_train_files, validation_data=itertools.cycle(validate_generator), validation_steps=n_validate_files )

Epoch 1/100
39/39 [==============================] - 50s 1s/step - loss: 0.9495 - accuracy: 0.5351 - val_loss: 0.7076 - val_accuracy: 0.7198
Epoch 2/100
39/39 [==============================] - 43s 1s/step - loss: 0.9330 - accuracy: 0.5349 - val_loss: 0.7108 - val_accuracy: 0.7112
Epoch 3/100
39/39 [==============================] - 50s 1s/step - loss: 0.9271 - accuracy: 0.5336 - val_loss: 0.7095 - val_accuracy: 0.6927
Epoch 4/100
39/39 [==============================] - 50s 1s/step - loss: 0.9233 - accuracy: 0.5348 - val_loss: 0.7071 - val_accuracy: 0.6681
Epoch 5/100
39/39 [==============================] - 68s 2s/step - loss: 0.9183 - accuracy: 0.5456 - val_loss: 0.7030 - val_accuracy: 0.6548
Epoch 6/100
39/39 [==============================] - 54s 1s/step - loss: 0.9218 - accuracy: 0.5413 - val_loss: 0.7109 - val_accuracy: 0.5780
Epoch 7/100
39/39 [==============================] - 54s 1s/step - loss: 0.9099 - accuracy: 0.5435 - val_loss: 0.7088 - val_accuracy: 0.4930
Epoch 8/100
3

KeyboardInterrupt: 